<a href="https://colab.research.google.com/github/Yumlembam/Kaggle-Distracted-Driver/blob/master/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# upload the credential of kaggle account
from google.colab import files
files.upload()

{}

In [0]:
#before importing the dataset we want to use this code
# The Kaggle API client expects this file to be in ~/.kaggle,
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
#import the dataset we want to use for our project
!kaggle competitions download -c state-farm-distracted-driver-detection

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
#Since we divide the test set, load the .h5 file accordingly
import h5py
with h5py.File('/gdrive/My Drive/10_test_array.h5', 'r') as hf:
    data_test = hf['test_dataset_10'][:]

In [0]:
# Creating place holder
import tensorflow as tf
from tensorflow.python.framework import ops
import math

In [0]:
saver = tf.train.import_meta_graph('/gdrive/My Drive/new_driver_state.ckpt.meta')
graph = tf.get_default_graph()
sess = tf.Session()
saver.restore(sess, "/gdrive/My Drive/new_driver_state.ckpt")
#pprint.pprint([v.name for v in tf.global_variables()])
#for op in graph.get_operations():
  #print(op.name)
X = graph.get_tensor_by_name("X_placeholder:0")
print(str(X))
Y = graph.get_tensor_by_name("Y_placeholder:0")
print(str(Y))
Z3= graph.get_tensor_by_name('fully_connected/BiasAdd:0')
#print(Z3.eval())
#predict_op = tf.argmax(Z3, 1,name="Prediction")
#correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))
# Calculate accuracy on the test set
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
#print(accuracy)
test_output = sess.run(Z3, feed_dict={X: data_test})
#train_accuracy = accuracy.eval({X: X_train, Y: y_train})
print("Train Accuracy:", test_output)

In [0]:
#Since we divide the test set save accordingly
np.savetxt("/gdrive/My Drive/10_output.csv", output, delimiter=",")

In [0]:
#Just to make sure notebook dosent cache any value
del data_test
del output
del test_output

Execute the cell below after completing execution of the graph on all the test dataset.

In [0]:
# combine all the output in one file
import csv
header = ['c0', 'c1','c2', 'c3','c4', 'c5','c6', 'c7','c8', 'c9']
fout=open("/gdrive/My Drive/All_output.csv","a")  
writer = csv.writer(fout, delimiter=',')
writer.writerow(header)
for num in range(1,11):
  f = open("/gdrive/My Drive/"+str(num)+"_output"+".csv","r+")
  for line in f:
    if line:
      fout.write(line)
  f.close()
fout.close()

In [16]:
import pandas as pd
all_output_df= pd.read_csv('/gdrive/My Drive/All_output.csv', header=0, sep=',', quotechar='"')
print(type(all_output_df.values))

<class 'numpy.ndarray'>


In [17]:
a = all_output_df.values

with tf.Session() as sess:
  prob = sess.run(tf.nn.softmax(a))
  print(prob)

[[1.46867931e-25 7.18464382e-39 2.52608797e-34 ... 5.67619656e-30
  1.04469012e-28 1.43764113e-17]
 [1.64854128e-06 1.29759154e-04 8.42613969e-02 ... 4.60423269e-09
  1.76797309e-07 7.28670068e-06]
 [1.02125005e-01 4.45429021e-01 4.70159956e-05 ... 2.18712314e-04
  7.70363031e-03 4.42536895e-01]
 ...
 [1.06141150e-03 9.94142669e-01 7.49012478e-06 ... 4.71007038e-04
  1.53726732e-03 4.63054093e-07]
 [4.87079556e-07 8.78558241e-14 4.25249118e-03 ... 2.70155768e-10
  3.25988361e-07 4.11545869e-11]
 [4.92267821e-14 5.12390338e-16 3.21948811e-20 ... 1.95859018e-12
  6.49135925e-06 4.24113164e-11]]


In [26]:
prob.shape

(79726, 10)

In [0]:
import numpy as np
np.savetxt("/gdrive/My Drive/prob1_output.csv", prob, delimiter=",")

In [0]:
import csv
header = ['c0', 'c1','c2', 'c3','c4', 'c5','c6', 'c7','c8', 'c9']
fout=open("/gdrive/My Drive/submission1_prob_output.csv","a")  
writer = csv.writer(fout, delimiter=',')
writer.writerow(header)
f = open("/gdrive/My Drive/prob1_output.csv","r+")
for line in f:
  if line:
    fout.write(line)
f.close()
fout.close()

In [31]:
output_df =pd.read_csv('/gdrive/My Drive/submission1_prob_output.csv', header=0, sep=',', quotechar='"')
print(output_df.shape)

(79726, 10)


In [32]:
#Merging the image name and predicted values
df_col_merged =pd.concat([test_df["img"], output_df], axis=1)
print(df_col_merged.head)
print(df_col_merged.shape)

<bound method NDFrame.head of                   img            c0            c1            c2            c3  \
0           img_1.jpg  1.468679e-25  7.184644e-39  2.526088e-34  4.451411e-22   
1          img_10.jpg  1.648541e-06  1.297592e-04  8.426140e-02  7.965629e-01   
2         img_100.jpg  1.021250e-01  4.454290e-01  4.701600e-05  8.144230e-06   
3        img_1000.jpg  4.546844e-13  8.613402e-01  2.301230e-12  7.873325e-12   
4      img_100000.jpg  1.530981e-08  1.605401e-19  4.659475e-12  2.087037e-02   
5      img_100001.jpg  3.531345e-30  2.891284e-23  2.385166e-41  1.000000e+00   
6      img_100002.jpg  5.871259e-15  1.000000e+00  7.084651e-15  1.382611e-09   
7      img_100003.jpg  6.312677e-03  3.574346e-08  3.255467e-11  1.700602e-03   
8      img_100004.jpg  3.322278e-14  5.235150e-10  2.839992e-11  2.702431e-09   
9      img_100005.jpg  1.103615e-18  2.185751e-07  1.704909e-12  5.504486e-14   
10     img_100007.jpg  1.100631e-07  1.116808e-12  1.356860e-11  2.697750e-12  